# Быстрое перемножение чисел

Умножение - с одной стороны это очень простая операция, которой люди овладели давным давно. Казалось бы, что ничего умнее, чем классический алгоритм перемножения столбиком не существует. Однако в 1960 Анатолий Карацуба изобрел алгоритм перемножения чисел быстрее, чем класический алгоритм.

# Классический алгоритм

Для понимания проблематики, сначала рассмотрим классический подход.

Во первых, для работы с большими числами нам потребуется другой способ хранить числа, так как классических типов данных нам, увы, не хватит. Для этого создадим класс, который будет представлять большие числа. Не будем сейчас задумываться об эффективность хранения данных, а также об излишних копированиях - это позволит сконцентрироваться на самом алгоритме.

Числа будем хранить как последовательность цифр в обратном порядке(так удобнее). Таким образом, например, сумма двух таких чисел - это просто поэлементная сумма двух массивов с последующей нормировкой и переносом цифр, которые были "в уме".

Классический алгоритм заключается в следующем:

* Перемножаем первое число на последнюю цифру второго (с нормировкой)
* Потом на вторую, но со двигом на 1
* Делаем так для каждой цифры второго числа
* Складываем полученные результаты

Таким образом, перемножение двух чисел длинной n цифр займет у нас n \* n операций.

In [1]:
#include <iostream>
#include <vector>
#include <iterator>
#include <tuple>

In [2]:
class Num {
public:
    Num() {}
    Num(const Num& other) : digits(other.digits) {} // конструктор копирования
    Num(std::string s){ // создание из строки
        digits.reserve(s.size());
        for(auto it = s.rbegin(); it!=s.rend(); it++) {
            digits.push_back( *it - '0' );
        }
    }
    Num(std::vector<int> v) : digits(v) {}
    ~Num(){}

    std::string to_s() { // превращениечисла в строку
        std::string res;
        res.reserve(digits.size());
        for(auto it = digits.rbegin(); it!=digits.rend(); it++) {
            res.push_back(static_cast<char>(*it + '0'));
        }
        return res;
    }
    
    static Num mul(Num a, Num b) { // перемножение столбиком
        std::vector<int> r(a.digits.size()+b.digits.size(), 0); // куда положим результат
        for(int i = 0; i < a.digits.size(); i++) { 
            for(int j = 0; j < b.digits.size(); j++){ // запускаем двойной цикл
                r[i+j] += a.digits[i] * b.digits[j]; // перемножение со двигом
                r[i+j+1] += r[i+j] / 10; // нормируем - переносим больше 10 в следующий разряд
                r[i+j] = r[i+j] % 10; // и оставляем только цифру
            }
        }
        if(r[r.size()-1] == 0) r.erase(r.end()-1); // убираем лишний 0
        return Num(r);
    }
    
private:
    std::vector<int> digits;
};

In [3]:
std::cout << Num::mul(Num("123"), Num("456")).to_s() << std::endl;

56088


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f7f2ceef6e0


In [4]:
std::cout << Num::mul(Num("123456"), Num("789101")).to_s() << std::endl;

97419253056


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f7f2ceef6e0


# Умножение Карацубы

Для того, чтобы добиться результата лучше чем $n^2$, воспользуемся стратегией "Раздлеляй и властвуй".

Пусть мы хотим перемножить X и Y. Тогда разделим оба числа на две части по половине (то есть например 123456 поделится на 123 И 456) - $X_1, X_2, Y_1, Y_2$. Заметим, что если во второй части n цифр, то тогда начальные числа выразятся следующим образом:

$ X = X_1 \cdot 10^n + X_2, \:\: Y = Y_1 \cdot 10^n + Y_2$

Рассмотрим их произведение

$X \cdot Y = (X_1 \cdot 10^n + X_2) \cdot (Y_1 \cdot 10^n + Y_2) = X_2 \cdot Y_2 + X_1 \cdot Y_1 \cdot 10^{2n} + (X_1 \cdot Y_2 + Y_1 \cdot X_2) \cdot 10^n$

Теперь небольшой математический трюк:

$ (X_1 + X_2) \cdot (Y_1 + Y_2) = X_2 \cdot Y_2 + X_1 \cdot Y_1 + (X_1 \cdot Y_2 + Y_1 \cdot X_2) $

А это значит, что 

$ X_1 \cdot Y_2 + Y_1 \cdot X_2 = (X_1 + X_2) \cdot (Y_1 + Y_2) - X_2 \cdot Y_2 - X_1 \cdot Y_1 $

А это ведь третье слагаемое в изначальной сумме.

Таким образом, если мы вычислим сделующие произведения: $ X_2 \cdot Y_2 , X_1 \cdot Y_1 , (X_1 + X_2) \cdot (Y_1 + Y_2)$, то мы сможем за линейное время восстановить искомое число - сложение производится за n операций, а домножение на $10^n$ - это всего лишь дописывание n нулей в конец числа.

Обратите внимание, что нам требуется всего 3 перемножения, вместо 4, которые были изначально. 

Остается вопрос - как нам вычислить эти три перемножения. Ответ - рекурсивно алгоритмом Карацубы, пока большие числа и классическим алгоритмом, как только числа станут маленькие (например, состоящие всего из 2 цифр).

Карацуба доказал, что данный метод, требующий всего 3 умножений вместо 4 на каждом уровне рекурсии требует всего $ \alpha \cdot n^{\log_2{3}} = \alpha \cdot n^{1.5849} $, что, как можно видеть, быстрее чем $\alpha \cdot n^2$ у классического алгоритма (альфа - какая-то константа).

In [5]:
class Num2 {
public:
    Num2() {}
    Num2(const Num2& other) : digits(other.digits) {}
    Num2(std::string s){
        digits.reserve(s.size());
        for(auto it = s.rbegin(); it!=s.rend(); it++) {
            digits.push_back( *it - '0' );
        }
    }
    Num2(std::vector<int> v) : digits(v) {}
    ~Num2(){}

    std::string to_s() {
        std::string res;
        res.reserve(digits.size());
        for(auto it = digits.rbegin(); it!=digits.rend(); it++) {
            res.push_back(static_cast<char>(*it + '0'));
        }
        return res;
    }

    Num2 cleared() { // для очистки от лишних нулей в начале числа
        auto it = digits.end()-1;
        for(; *it == 0;) it--;
        digits.erase(it+1, digits.end());
        return Num2(digits);

    }

    std::tuple<Num2, Num2> partition(int n) { // разобьем число на две части
        return std::make_tuple(
                Num2(std::vector<int>(digits.end() - n, digits.end())),
                Num2(std::vector<int>(digits.begin(), digits.end() - n))
        );
    }

    Num2 pow10(int n) { // домножение на 10^n
        std::vector<int> r(n, 0);
        std::copy(digits.begin(), digits.end(), std::back_inserter(r));
        return Num2(r);
    }


    static Num2 mul(Num2 a, Num2 b) { // классический алгоритм. Он нам еще пригодится
        std::vector<int> r(a.digits.size()+b.digits.size(), 0);
        for(int i = 0; i < a.digits.size(); i++) {
            for(int j = 0; j < b.digits.size(); j++) {
                r[i+j] += a.digits[i] * b.digits[j];
                r[i+j+1] += r[i+j] / 10;
                r[i+j] = r[i+j] % 10;
            }
        }
        if(r[r.size()-1] == 0) r.erase(r.end()-1);
        return Num2(r);
    }

    static Num2 sum(Num2 a, Num2 b) { // сумма двух чисел
        int m = std::max(a.digits.size(), b.digits.size());
        std::vector<int> r(m+1, 0);
        a.digits.resize(m); b.digits.resize(m);
        for(int i = 0; i < m; i++) {
            r[i] += a.digits[i]+b.digits[i];
            r[i+1] += r[i] / 10;
            r[i] = r[i] % 10;
        }
        if(r[r.size()-1] == 0) r.erase(r.end()-1);
        return Num2(r);
    }

    static Num2 sub(Num2 a, Num2 b) { // a - b
        int m = std::max(a.digits.size(), b.digits.size());
        std::vector<int> r(m+1, 0);
        a.digits.resize(m); b.digits.resize(m);
        for(int i = 0; i < m; i++) {
            r[i] += a.digits[i]-b.digits[i];
            r[i+1] -= r[i] < 0 ? 1 : 0;
            r[i] = r[i] < 0 ? 10 + r[i] : r[i];
        }
        if(r[r.size()-1] == 0) r.erase(r.end()-1);
        return Num2(r);
    }

    static Num2 karazuba(Num2 a, Num2 b) { // перемножение карацубы
        if(a.digits.size() < 3 or b.digits.size() < 3) {
            return Num2::mul(a, b); // если числа маленькие, что можно воспользоваться классическим алгоритмом
        }
        int m = std::max(a.digits.size(), b.digits.size());
        a.digits.resize(m); b.digits.resize(m); // допишем незначащие нули в начало каждого числа для удобства
        // теперь оба числа имеют формально размер m цифр

        Num2 x1, x2, y1, y2; 
        std::tie(x1, x2) = a.partition(m/2); // разобьем наши числа по половине
        std::tie(y1, y2) = b.partition(m/2);


        Num2 x1y1 = Num2::karazuba(x1, y1); // вычислим рекурсивно все три необходимые произведения
        Num2 x2y2 = Num2::karazuba(x2, y2);
        Num2 x1y2_y1x2 = Num2::karazuba(Num2::sum(x1, x2), Num2::sum(y1, y2));
        x1y2_y1x2 = Num2::sub(x1y2_y1x2, Num2::sum(x1y1, x2y2)); // восстановим третье слагаемое в основной сумме

        return Num2::sum(x2y2, Num2::sum(x1y1.pow10( 2*(m-m/2) ), x1y2_y1x2.pow10((m-m/2)))).cleared();
        // вернем нужную сумму
    }

private:
    std::vector<int> digits;
};


In [7]:
std::cout << Num2::karazuba(Num2("123456"), Num2("789101")).to_s() << std::endl;

97419253056


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f7f2ceef6e0


In [8]:
std::cout << Num2::karazuba(Num2("123456789"), Num2("7891011121")).to_s() << std::endl;

974198894961950469


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f7f2ceef6e0


Можете проверить вычисления на кальтуляторе - они все корректные.